# 머신러닝 실습(회귀)
- section 1 : 문제정의
- section 2 : 라이브러리 및 데이터 불러오기
- section 3 : 탐색적데이터분석(EDA)
- section 4 : 데이터 전처리
- section 5 : 검증 데이터 나누기
- section 6 : 머신러닝 학습 및 평가
- section 7 : 예측 및 결과 파일 생성

## 1. 문제정의
- 10개의 아울렛 매장에서 1,500여개 제품에 대한 판매 데이터를 수집했다.
- 예측 모델을 만들고 아울렛 특정 매장에서 각 제품의 판매금액을 예측하시오.

    - 평가 기준은 RMSE로 평가
    - label(target)은 판매금액(Item_Outlet_Sale)
    - 제출 파일은 예측값만 result.csv 파일로 생성해 제출(컬럼명 : pred , 1개)

## 2. 라이브러리 및 데이터 불러오기

In [1]:
import pandas as pd
train = pd.read_csv('./ch4_data/train.csv')
test = pd.read_csv('./ch4_data/test.csv')


## 3. 탐색적 데이터 분석(EDA)
- 데이터의 크기를 확인한다.

- train 데이터 (6818, 12)
- test 데이터(1705, 11)

In [2]:
print(train.shape)
print(test.shape)

(6818, 12)
(1705, 11)


### 데이터 샘플 확인
- 카테고리(문자)와 숫자 컬럼이 혼합되어있는 것을 확인
- 마지막 컬럼 Item_outlet_Sales는 target(label) 컬럼인데, 금액이고 소수점있는 숫자.

In [3]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCR06,12.500,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,FDW11,12.600,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,FDH32,12.800,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890
3,FDL52,6.635,Regular,0.046351,Frozen Foods,37.4506,OUT017,2007,NaN,Tier 2,Supermarket Type1,1176.4686
4,FDO09,13.500,Regular,0.125170,Snack Foods,261.4910,OUT013,1987,High,Tier 3,Supermarket Type1,3418.8830


### 데이터 자료형 확인
- float형 4개
- int형 1개
- object 형 7개

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                5656 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                4878 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
 11  Item_Outlet_Sales          6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB


### 기술 통계 확인
- describe()
---
- median(50%)>mean인 경우 왼쪽 꼬리가 긴 분포
- median(50%)<mean인 경우 오른쪽 꼬리가 긴 분포
---
- Item_Visibility, Item_Outlet_Sales 컬럼은 Median(50%)<mean 인 데이터모양이 오른쪽꼬리가 긴 분포모양

In [5]:
train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,5656.000000,6818.000000,6818.000000,6818.000000,6818.000000
mean,12.872703,0.066121,140.419533,1997.885890,2190.941459
std,4.651034,0.051383,62.067861,8.339795,1706.131256
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026914,93.610050,1987.000000,836.577700
50%,12.600000,0.053799,142.448300,1999.000000,1806.648300
75%,17.000000,0.095273,185.060150,2004.000000,3115.944000
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


### Object 컬럼의 unique 개수 파악
- 3개부터 많게는 1554개까지 분포
- Item_Identifier은 1,554개
---
- unique의 개수에 따라 원-핫 인코딩을 사용하거나 레이블 인코딩을 사용하기도 한다
- 1,5554개를 원-핫 인코딩을 했을 때는 컬럼수가 엄청나게 늘어나고 대부분의 값이 0으로 대체되며 낭비가 심하다.

In [6]:
train.describe(include='O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,6818,6818,6818,6818,4878,6818,6818
unique,1554,5,16,10,3,3,4
top,FDW26,Low Fat,Snack Foods,OUT046,Medium,Tier 3,Supermarket Type1
freq,9,4092,963,763,2228,2664,4474


### test 데이터셋 분포 확인
- Item_Identifier를 제외하고 모두 train 데이터 셋과 unique 개수가 동일

In [7]:
test.describe(include='O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,1705,1705,1705,1705,1235,1705,1705
unique,1077,5,16,10,3,3,4
top,FDG33,Low Fat,Fruits and Vegetables,OUT013,Medium,Tier 3,Supermarket Type1
freq,4,997,272,207,565,686,1103


### 결측치 확인
- train Data
    - Item_Weight 1162개 
    - Outlet_Size는 1940개
---
- test Data
    - Item_Weight 301개 
    - Outlet_Size는 470개

In [8]:
train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1162
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1940
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [9]:
test.isnull().sum()

Item_Identifier                0
Item_Weight                  301
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                  470
Outlet_Location_Type           0
Outlet_Type                    0
dtype: int64

## 4. 데이터 전처리


### 인코딩처리를 위해 Object 컬럼 선택


In [10]:
list(train.columns[train.dtypes=='object'])

['Item_Identifier',
 'Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [11]:
cols = [
    'Item_Fat_Content',
    'Item_Type',
    'Outlet_Identifier',
    'Outlet_Size',
    'Outlet_Location_Type',
    'Outlet_Type'
]

cols

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

### target 컬럼 변수에 옮긴 후 데이터 인코딩

In [12]:
target = train.pop('Item_Outlet_Sales')
print(train.shape, test.shape)
df = pd.concat([train,test])
print(df.shape)

(6818, 11) (1705, 11)
(8523, 11)


### 레이블 인코딩 진행


In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cols:
    df[col] = le.fit_transform(df[col])

In [14]:
# 결과 확인
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,NCR06,12.500,1,0.006760,9,42.8112,1,1987,0,2,1
1,FDW11,12.600,1,0.048741,1,60.4194,1,1987,0,2,1
2,FDH32,12.800,1,0.075997,6,97.1410,1,1987,0,2,1
3,FDL52,6.635,2,0.046351,5,37.4506,2,2007,3,1,1
4,FDO09,13.500,2,0.125170,13,261.4910,1,1987,0,2,1


### 인코딩 결과 분리 

In [15]:
train = df.iloc[:len(train)].copy()
test = df.iloc[len(train):].copy()
train.shape, test.shape

((6818, 11), (1705, 11))

### 결측치는 최솟값과, 최빈값으로 채우기


In [16]:
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0])


### 전처리 중 스케일링, 이상치 등은 베이스라인을 만들기 위한 필수 요소가 아니라 선택이므로 생략

In [17]:
# item_id 삭제
print(train.shape, test.shape)

train.drop('Item_Identifier',axis=1,inplace=True)
test.drop('Item_Identifier',axis=1,inplace=True)
print(train.shape, test.shape)

(6818, 11) (1705, 11)
(6818, 10) (1705, 10)


## 5. 검증 데이터 나누기
- 1. X 데이터에서 타깃 변수 제외
- 2. y 데이터는 시리즈 형태
- 3. random_state 고정


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((5454, 10), (1364, 10), (5454,), (1364,))

## 6. 머신러닝 학습 및 평가
- 회귀모델의 평가지표 중 RMSE를 평가지표로 확인했다.
- 학습 과정이므로 RMSE 뿐만 아니라 MSE,MAE,R2로 평가


In [19]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# from sklearn.metrics import root_mean_squared_error


### 1. 선형 회귀
- 기본적인 회귀 모델인 선형 회귀 모델로 학습하고 X_val을 예측한 결과값을 y_val과 비교 평가해보기
- 선형회귀 모델은 Random_state 하이퍼파라미터 값이 없다.


In [20]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)

result = mean_squared_error(y_val,y_pred)
print("MSE :" ,result)

result = mean_absolute_error(y_val,y_pred)
print("MAE :" ,result)

result = r2_score(y_val,y_pred)
print("R2 :" ,result)

# result = root_mean_squared_error(y_val,y_pred)
# print("RMES :" ,result)

MSE : 1282923.0729833886
MAE : 865.1968401416268
R2 : 0.5058168396924846


### 2. 랜덤 포레스트
- 랜덤포레스트는 분류도 있고 회귀도 있다.
- 회귀로 사용할 때는 RandomForestRegressor를 불러와 사용


In [21]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_val)

result = mean_squared_error(y_val,y_pred)
print("MSE : ", result)

result = mean_absolute_error(y_val,y_pred)
print("MAE : ", result)

result = r2_score(y_val,y_pred)
print("R2 : ", result)

# result = root_mean_squared_error(y_val,y_pred)
# print("RMES :" ,result)

MSE :  1101802.8117346708
MAE :  747.2971172067448
R2 :  0.5755845326933362


### LightGBM
- LightGBM 도 랜덤 포레스트와 같이 회귀모델이있다.

In [22]:
import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=0, verbose=1)
model.fit(X_train,y_train)
y_pred = model.predict(X_val)

result = mean_squared_error(y_val,y_pred)
print("MSE : ", result)

result = mean_absolute_error(y_val,y_pred)
print("MAE : ", result)

result = r2_score(y_val,y_pred)
print("R2 : ", result)

# result = root_mean_squared_error(y_val,y_pred)
# print("RMES :" ,result)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000409 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.

c:\TeamGit\-BigDataAnalyst\.venv\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\TeamGit\-BigDataAnalyst\.venv\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\user\AppData\Local\Programs\Python\Python38\lib\subprocess.py", line 493, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\user\AppData\Local\Programs\Python\Python38\lib\subprocess.py", line 858, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\user\AppData\Local\Programs\Python\Python38\lib\subprocess.py", line 1311, in _execute_child
    hp, ht, pid, tid = _winapi.Cr


[LightGBM] [Info] Total Bins 787
[LightGBM] [Info] Number of data points in the train set: 5454, number of used features: 10
[LightGBM] [Info] Start training from score 2202.546849
MSE :  1115654.3482227568
MAE :  736.6367966578568
R2 :  0.5702489079618556


## 7. 예측 및 결과 파일 생성
- test 데이터를 예측한다.

In [23]:
pred = model.predict(test)
pred

array([1226.50504175,  830.18920951, 1788.76644309, ..., 3666.34876099,
        990.65125496, 1224.1960178 ])

In [24]:
submit=pd.DataFrame({'pred' :pred})
submit.to_csv('result.csv',index=False)